In [1]:
import os
from PIL import Image
import time
import sys
import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file
image_folder = '../../DLSP20Dataset/data'
annotation_csv = '../../DLSP20Dataset/data/annotation.csv'


# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file
#image_folder = '../../data'
#annotation_csv = '../../data/annotation.csv'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#device = 'cpu'

#device = "cpu"

torch.cuda.is_available()

# function to count number of parameters
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

def order_points(pts):
    from scipy.spatial import distance as dist
    import numpy as np
    
    xSorted = pts[np.argsort(pts[:, 0]), :]

    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    return np.array([tl, tr, br, bl]).astype(float)

def arrange_box(x1,y1):
    box=np.array(list(zip(x1,y1)))
    box=order_points(box)
    return box

def iou(box1, box2):
    from shapely.geometry import Polygon
    a = Polygon(torch.t(box1)).convex_hull
    b = Polygon(torch.t(box2)).convex_hull
    
    return a.intersection(b).area / a.union(b).area

#def iou(xy1,xy2):
#    
#    from shapely.geometry import Polygon
#    
#    boxA = Polygon(arrange_box(xy1[0],xy1[1])).buffer(1e-9)
#    boxB = Polygon(arrange_box(xy2[0],xy2[1])).buffer(1e-9)
#    
#    try:
#        return boxA.intersection(boxB).area / boxA.union(boxB).area
#    except:
#        print('Box 1:',xy1[0],xy1[1])
#        print('Box 2:',xy2[0],xy2[1])
#        sys.exit(1)

def map_to_ground_truth(overlaps, print_it=False):
    prior_overlap, prior_idx = overlaps.max(1)
    if print_it: print(prior_overlap)
#     pdb.set_trace()
    gt_overlap, gt_idx = overlaps.max(0)
    gt_overlap[prior_idx] = 1.99
    for i,o in enumerate(prior_idx): gt_idx[o] = i
    return gt_overlap,gt_idx

def calculate_overlap(target_bb, predicted_bb):
    overlaps = torch.zeros(target_bb.size(0),predicted_bb.size(0))

    for j in range(overlaps.shape[0]):
        for k in range(overlaps.shape[1]):
            overlaps[j][k] = iou(target_bb[j],predicted_bb[k])
            
    return overlaps

def one_hot_embedding(labels, num_classes):
    return torch.eye(num_classes)[labels.data.cpu()]

from skimage import draw
import numpy as np

def poly2mask(vertex_row_coords, vertex_col_coords, shape):
    fill_row_coords, fill_col_coords = draw.polygon(vertex_row_coords, vertex_col_coords, shape)
    mask = torch.zeros(shape, dtype=np.bool)
    mask[fill_row_coords, fill_col_coords] = True
    return mask

def convert_to_binary_mask(corners, shape=(800,800)):
    point_squence = torch.stack([corners[:, 0], corners[:, 1], corners[:, 3], corners[:, 2], corners[:, 0]])
    x,y = point_squence.T[0].detach() * 10 + 400, -point_squence.T[1].detach() * 10 + 400
    new_im = poly2mask(y, x, shape)
    return new_im

def create_conf_matrix(target, pred, debug=True):
    import sys
    
    target = target.reshape(-1)
    pred = pred.reshape(-1)
    
    if debug:
        print('Target values:', target.unique())
        print('Predicted values:', pred.unique())
        print('Target shape:', target.shape)
        print('Predicted shape:', pred.shape)
    
    nb_classes = max(target.unique())
    if len(pred.unique()) > (nb_classes+1) :
        print('More predicted classes than true classes')
        sys.exit(1)
        
    conf_matrix = torch.zeros(nb_classes+1, nb_classes+1)
    for t, p in zip(target, pred):
        conf_matrix[t, p] += 1
    
    return conf_matrix

def create_conf_matrix2(target, pred, debug=True):
    import sys
    
    target = target.reshape(-1).cpu().numpy()
    pred = pred.reshape(-1).cpu().numpy()
    
        
    conf_matrix = torch.from_numpy(confusion_matrix(target, pred)).to(device)
    
    return conf_matrix

def classScores(conf_matrix):
    print('Confusion matrix\n', conf_matrix)
    TP = conf_matrix.diag()
    TN = torch.zeros_like(TP)
    FP = torch.zeros_like(TP)
    FN = torch.zeros_like(TP)
    for c in range(conf_matrix.size(0)):
        idx = torch.ones(conf_matrix.size(0)).byte()
        idx[c] = 0
        # all non-class samples classified as non-class
        TN[c] = conf_matrix[idx.nonzero()[:, None], idx.nonzero()].sum() #conf_matrix[idx[:, None], idx].sum() - conf_matrix[idx, c].sum()
        # all non-class samples classified as class
        FP[c] = conf_matrix[idx, c].sum()
        # all class samples not classified as class
        FN[c] = conf_matrix[c, idx].sum()

        print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
            c, TP[c], TN[c], FP[c], FN[c]))
        
    return TP, TN, FP, FN

def IOU(bbox1, bbox2):
    '''Calculate overlap between two bounding boxes [x, y, w, h] as the area of intersection over the area of unity'''
    x1, y1, w1, h1 = bbox1[0], bbox1[1], bbox1[2], bbox1[3]  # TODO: Check if its more performant if tensor elements are accessed directly below.
    x2, y2, w2, h2 = bbox2[0], bbox2[1], bbox2[2], bbox2[3]

    w_I = min(x1 + w1, x2 + w2) - max(x1, x2)
    h_I = min(y1 + h1, y2 + h2) - max(y1, y2)
    w_I = max(w_I, 0)  # set w_I and h_I zero if there is no intersection
    h_I = max(h_I, 0)
    I = w_I * h_I

    U = w1 * h1 + w2 * h2 - I

    return I / U

def dist(bbox1, bbox2):
    return torch.sqrt(torch.sum(torch.square(bbox1[:2] - bbox2[:2])))

# Dataset

# You shouldn't change the unlabeled_scene_index
# The first 106 scenes are unlabeled
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)

train_scene_index = np.random.choice(labeled_scene_index, int(np.ceil(0.1*len(labeled_scene_index))))

#test_scene_index = labeled_scene_index[np.isin(labeled_scene_index, train_scene_index,invert=True)]
test_scene_index = train_scene_index


transform=torchvision.transforms.Compose([torchvision.transforms.Resize((100,100)),
                                          torchvision.transforms.ToTensor(),
                              #torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])


# Labeled dataset

batch_size = 1

NUM_SAMPLE_PER_SCENE = 126
NUM_IMAGE_PER_SAMPLE = 6
image_names = [
    'CAM_FRONT_LEFT.jpeg',
    'CAM_FRONT.jpeg',
    'CAM_FRONT_RIGHT.jpeg',
    'CAM_BACK_LEFT.jpeg',
    'CAM_BACK.jpeg',
    'CAM_BACK_RIGHT.jpeg',
    ]

def boxToHWC(boxes1):
    boxes1_max_x = boxes1[:, 0].max(dim=1)[0]
    boxes1_min_x = boxes1[:, 0].min(dim=1)[0]
    boxes1_max_y = boxes1[:, 1].max(dim=1)[0]
    boxes1_min_y = boxes1[:, 1].min(dim=1)[0]
    
    x = (boxes1_max_x + boxes1_min_x)/2
    y = (boxes1_max_y + boxes1_min_y)/2
    h = (boxes1_max_y - boxes1_min_y)
    w = (boxes1_max_x - boxes1_min_x)
    
    return torch.stack((x,y,h,w)).T


def imgToHWC(img):
    height, width = img.shape[:2]     
    x1 = int(row['XMin'] * width)     
    x2 = int(row['XMax'] * width)     
    y1 = int(row['YMin'] * height)     
    y2 = int(row['YMax'] * height)
    return (x1+x2)/2, (y1+y2)/2, height, width


def transformTargetstoHWC(targets):
    bounding_box = targets["bounding_box"]
    max, ind = torch.max(bounding_box, dim=3)
    min, ind = torch.min(bounding_box, dim=3)
    c = (400 + ((max + min)/2) * 10) / 1.92
    w = (max - min) * 10 / 1.92
    target = torch.cat((c, w), dim=2)
    category = targets["category"]
    targets = torch.cat((target, category.unsqueeze(2).double()), dim=2).float()
    target_lengths = torch.tensor(targets.numpy().shape[:2])
    return targets, target_lengths


# The dataset class for labeled data.
class LabeledSSDDataset(torch.utils.data.Dataset):    
    def __init__(self, image_folder, annotation_file, scene_index, transform, extra_info=True):
        """
        Args:
            image_folder (string): the location of the image folder
            annotation_file (string): the location of the annotations
            scene_index (list): a list of scene indices for the unlabeled data 
            transform (Transform): The function to process the image
            extra_info (Boolean): whether you want the extra information
        """
        
        self.image_folder = image_folder
        self.annotation_dataframe = pd.read_csv(annotation_file)
        self.scene_index = scene_index
        self.transform = transform
        self.extra_info = extra_info
    
    def __len__(self):
        return self.scene_index.size * NUM_SAMPLE_PER_SCENE

    def __getitem__(self, index):
        scene_id = self.scene_index[index // NUM_SAMPLE_PER_SCENE]
        sample_id = index % NUM_SAMPLE_PER_SCENE
        sample_path = os.path.join(self.image_folder, f'scene_{scene_id}', f'sample_{sample_id}') 

        images = []
        for image_name in image_names:
            image_path = os.path.join(sample_path, image_name)
            image = Image.open(image_path)
            images.append(self.transform(image))
        image_tensor = torch.stack(images)

        data_entries = self.annotation_dataframe[(self.annotation_dataframe['scene'] == scene_id) & (self.annotation_dataframe['sample'] == sample_id)]
        corners = data_entries[['fl_x', 'fr_x', 'bl_x', 'br_x', 'fl_y', 'fr_y','bl_y', 'br_y']].to_numpy()
        categories = data_entries.category_id.to_numpy()
        
        ego_path = os.path.join(sample_path, 'ego.png')
        ego_image = Image.open(ego_path)
        ego_image = torchvision.transforms.functional.to_tensor(ego_image)
        #road_image = convert_map_to_road_map(ego_image)
        
        target = {}
        target['bounding_box'] = torch.as_tensor(corners).view(-1, 2, 4)
        target['category'] = torch.as_tensor(categories).view(1,-1)
        
        target['chw'] = boxToHWC(target['bounding_box']).unsqueeze(0).double()


        return image_tensor, target['chw'], target['category']
        

# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
labeled_trainset = LabeledSSDDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )

trainloader = torch.utils.data.DataLoader(labeled_trainset, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=0, 
                                          collate_fn=collate_fn)

labeled_testset = LabeledSSDDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=test_scene_index,
                                  transform=transform,
                                  extra_info=True
                                 )

testloader = torch.utils.data.DataLoader(labeled_testset, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          num_workers=0, 
                                          collate_fn=collate_fn)

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.LeakyReLU(negative_slope=0.1),
        nn.BatchNorm2d(out_channels),
        #nn.Dropout(0.5),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.LeakyReLU(negative_slope=0.1),
        nn.BatchNorm2d(out_channels),
        #nn.Dropout(0.5)
    ) 


class CNN(nn.Module):
    def __init__(self, d=650, output_size=4):
        super().__init__()
        self.dconv_down1 = double_conv(3, 16)
        self.dconv_down2 = double_conv(16, 32)
        self.dconv_down3 = double_conv(32, 48)
        self.dconv_down4 = double_conv(48, 64)        
        self.maxpool = nn.MaxPool2d(2)
        self.linear1 = nn.Linear(6*64*12*12,4*88) ## 88 boxes
        self.linear2 = nn.Linear(6*64*12*12,88) ## 88 boxes
        
    def forward_once(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)


        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)  

        
        x = self.dconv_down4(x)
        return x
    
    def forward(self,x):
        x = [self.forward_once(y) for y in x]
        x = torch.cat(x,axis=1)
        x1 = self.linear1(x.reshape(-1,6*64*12*12))
        x2 = self.linear2(x.reshape(-1,6*64*12*12))
        return (x1,F.sigmoid(x2))

In [2]:
model = CNN()

mse = nn.MSELoss()

learning_rate = 3e-4

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_loss = 1000000000000000
num_epochs = 10
threshold = 0.5

In [ ]:
num_objects = 88

for epoch in range(0, num_epochs):
    model.train()
    train_loss = {}
    train_loss['boxes'] = 0
    train_loss['probs'] = 0
    train_loss['total'] = 0
    for (x, bb_true, class_true) in trainloader:
        exp = torch.stack(bb_true)
        pred, probs = model(torch.stack(x).reshape(6,-1,3,100,100))
        pred = pred.reshape(1,-1,4)
        probs = probs.reshape(1,-1,88)
        
        num_true = exp.size(2)
        pred = pred.reshape(num_objects, -1)
        exp = exp.reshape(num_true, -1)
        pred_bboxes = pred[:, :4]
        exp_bboxes = exp[:, :4]
        # TODO: Try flipping array and see if results differ.
        ious = np.zeros((num_true, num_objects))
        for i, exp_bbox in enumerate(exp_bboxes):
            for j, pred_bbox in enumerate(pred_bboxes):
                ious[i, j] = IOU(exp_bbox, pred_bbox)

        exp_idx, pred_idx = linear_sum_assignment(-ious)

        loss1 = mse(exp[exp_idx],pred[pred_idx])
        train_loss['boxes'] += loss1.item()
            
        label = torch.zeros(88)
        label[pred_idx] = 1
        loss2 = mse(probs.reshape(-1), label.reshape(-1))
        train_loss['probs'] = loss2.item()
        
        loss = loss1 + loss2
        
        train_loss['total'] = loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('====> Epoch: {} Average loss (boxes): {} / {}'.format(
    epoch,train_loss['boxes'],len(trainloader.dataset)))
    print('====> Epoch: {} Average loss (probs): {} / {}'.format(
    epoch,train_loss['probs'],len(trainloader.dataset)))
    print('====> Epoch: {} Average loss (total): {} / {}'.format(
    epoch,train_loss['total'],len(trainloader.dataset)))
    
    with torch.no_grad():
        model.eval()
        test_loss = {}
        test_loss['boxes'] = 0
        test_loss['probs'] = 0
        test_loss['total'] = 0
        for (x, bb_true, class_true) in testloader:
            exp = torch.stack(bb_true)
            pred, probs = model(torch.stack(x).reshape(6,-1,3,100,100))
            pred = pred.reshape(1,-1,4)
            probs = probs.reshape(1,-1,88)
            num_preds = exp.size(2)
            pred = pred.reshape(num_objects, -1)
            exp = exp.reshape(num_preds, -1)
            pred_bboxes = pred[:, :4]
            exp_bboxes = exp[:, :4]

            pred_idx = (probs>threshold).squeeze()
            print(pred_idx)
            predicted_boxes = pred[pred_idx]
            print(predicted_boxes)
            predicted_num_objects = torch.sum(probs>threshold)
            ious = np.zeros((num_preds, predicted_num_objects))
            for i, exp_bbox in enumerate(exp_bboxes):
                for j, pred_bbox in enumerate(predicted_boxes):
                    ious[i, j] = IOU(exp_bbox, pred_bbox)
                    
            print(np.mean(ious))

            

====> Epoch: 0 Average loss (boxes): 70111.21628114417 / 378
====> Epoch: 0 Average loss (probs): 0.206088587641716 / 378
====> Epoch: 0 Average loss (total): 162.36143132149786 / 378
tensor([False,  True,  True, False, False,  True, False, False, False, False,
         True, False, False,  True, False,  True, False,  True, False, False,
        False, False,  True, False,  True,  True,  True, False, False, False,
        False, False, False,  True,  True, False, False,  True, False, False,
        False, False, False, False, False, False, False,  True,  True, False,
         True, False,  True, False,  True, False, False,  True, False, False,
        False, False,  True, False, False,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        False,  True,  True,  True, False,  True,  True,  True])
tensor([[ -5.5389,  11.0793,   6.3388,   3.1887],
        [ -9.5666,  -9.6823,   7.9729,  10.0772],
        [-11.1722,   6.7030

0.00026484904944194376
tensor([False,  True, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False, False,
        False, False,  True, False, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True, False,
        False, False, False,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[-3.7052e+00,  5.1343e+00,  1.3586e+00,  2.3443e+00],
        [-1.0422e+01,  3.4988e+00,  1.2493e+00, -5.4256e+00],
        [ 2.4630e+01,  2.6124e+01, -2.1825e+01,  8.6610e-01],
        [-2.1264e+01,  1.7464e+01,  1.3428e+01, -3.8163e+00],
        [ 4.8040e+00,  7.9741e+00,  1.49

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 24.5794,  27.4500, -24.5926,   2.3097],
        [  0.3178,   0.3797,   0.9333,  -0.6759],
        [  4.1707,  -1.6997,  -9.4266,   3.2188],
        [  1.5929,   1.2718,  -1.9473,  10.9097],
        [ 13.8282,   9.5813,  -5.3428,   6.7422],
        [  3.1898,  21.7509,   2.7260,   9.0847],
        [ 1

0.00010210090587959851
tensor([False,  True, False, False, False, False, False, False, False, False,
        False, False,  True, False, False,  True, False, False, False, False,
        False, False,  True, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True, False,
        False, False, False, False,  True, False, False, False,  True,  True,
        False, False, False, False, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         True, False,  True,  True,  True,  True,  True,  True])
tensor([[-7.4742e+00, -1.1354e+00,  6.9547e+00,  4.0055e+00],
        [-2.0230e-01,  5.5196e-01,  9.2244e-02, -4.0826e-01],
        [-4.6432e+00,  1.7075e-01, -2.1027e+00, -3.7376e+00],
        [ 1.0195e+01,  4.4591e+00, -1.6433e+00, -1.9489e-01],
        [ 9.4726e-03, -2.4440e-01,  3.60

0.0016341423186402493
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[  1.3666,   2.0069,   4.1458,  -0.6017],
        [-25.4510,  32.0224,   4.8488,   2.1845],
        [ 12.1799,  15.5598,   9.5758,   3.1747],
        [ -6.0057,  12.3430,  12.9364,  10.0891],
        [ -9.5853,   6.0465,  -1.9889,   4.4008],
        [ 17.6302,  20.1771,  -2.2055, 

...

0.0006765978181570696
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[  9.9683,  -5.5992,  -1.8017,   5.6236],
        [ -5.0755,  31.4389,   1.6594,  22.0503],
        [ 21.8430,   2.6636,   0.9525,   3.5125],
        [ -8.3877,   7.7595,  16.3587,   2.4231],
        [ -6.1297,   6.1632,  -3.2616,   3.7218],
        [ -1.1145,  15.0988,  -1.5380, 

0.0010042104261706309
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[  6.3558,  -8.8456,   3.9501,   3.7906],
        [  1.0504,  -2.9251,  -0.8461,   5.5743],
        [-16.9550,  18.4358,   1.4717,   3.8995],
        [ 15.6811,  10.8286,  -8.2377,   8.6586],
        [ -4.7766,   6.1057,   7.9177,   4.1571],
        [ -4.2583,   6.1952,  -5.0336, 

0.0015814380781223503
tensor([False,  True, False, False, False, False, False, False, False, False,
         True, False,  True, False, False,  True, False, False, False, False,
        False, False,  True, False, False, False,  True, False,  True, False,
        False,  True, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False,  True, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True, False,
         True,  True,  True,  True,  True,  True, False,  True,  True,  True,
         True,  True,  True,  True, False,  True,  True,  True])
tensor([[ -7.6017,  11.8183,   9.5444,  10.3724],
        [ -5.4582, -11.8982,  -3.2838,   9.6624],
        [ -0.1494,  -0.3137,   0.6310,  -0.1163],
        [ -5.2877,  -1.2737,   0.7873,  -1.9969],
        [ 30.3677,  36.4751, -19.9192,  12.7546],
        [ -0.7110,   0.2014,  -0.5082, 

0.001620386758287772
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False, False,
         True,  True, False,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[  5.9286,  -5.0145,   0.9565,   2.0764],
        [ 17.8319,  10.4078,   1.1437,   8.1121],
        [-11.5729,   3.5437,   7.2121,   3.8288],
        [  1.1666,   2.3194,  -0.8764,   6.0627],
        [  0.3444,  -7.4812,   1.9534,   5.0339],
        [  0.9988,  -6.1179,  12.2800,  

0.0005555567554307694
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True])
tensor([[  7.9616,  -0.6211, -10.5354,   2.1866],
        [ 26.7168,   5.4373,   6.2671,   8.6813],
        [ -4.1386,  12.2372,   6.1000,   0.5815],
        [ 12.7400,  16.3182,  -6.1244,   7.3427],
        [ -6.9010,  21.7176,   3.4802,  14.7980],
        [ -3.3477,   5.7192,  -0.2641, 

In [ ]:
pred_bboxes

In [ ]:
exp